In [7]:
import random
import math
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

def create_defense():
    x = random.uniform(-0.45, 0.45)
    y = random.uniform(-0.45, 0.45)
    defense = np.array([x, y])

    return defense

def create_attack(defense):
    x_side_left = random.uniform(-0.95, (defense[0] - 0.02) - 0.1)
    x_side_right = random.uniform((defense[0] + 0.02) + 0.1, 0.95)
    y_below = random.uniform((defense[1] - 0.02) - 0.1, -0.95)
    y_above = random.uniform((defense[1] + 0.02) + 0.1, 0.95)
    x_inclusive = random.uniform(-0.95, 0.95)
    y_inclusive = random.uniform(-0.95, 0.95)
    y_below_x_inclusive = np.array([x_inclusive, y_below])
    y_above_x_inclusive = np.array([x_inclusive, y_above])
    x_left_y_inclusive = np.array([x_side_left, y_inclusive])
    x_right_y_inclusive = np.array([x_side_right, y_inclusive])

    attack = random.choice([y_below_x_inclusive, y_above_x_inclusive, x_left_y_inclusive, x_right_y_inclusive])
    return attack 

def calc_angle(defense, attack):

    # create an adjacent point of the form (attack_x, defense_y)
    adjacent_point = np.array([attack[0], defense[1]])

    # calculate the distance between the adjacent point and the defense, attack points
    adj_point_defense_len = abs(defense[0] - adjacent_point[0]) 
    adj_point_attack_len = abs(attack[1] - adjacent_point[1])

    # calculate the angle, using soh cah toa, where adj_point_defense_len is the adjacent side and adj_point_attack_len is the opposite side
    theta = np.arctan(adj_point_attack_len / adj_point_defense_len)
    
    if attack[0] > defense[0]:
        if attack[1] > defense[1]:
            theta = theta # 1st quadrant
        else: 
            theta = (2*math.pi) - theta # 360 - theta
    else:
        if attack[1] > defense[1]:
            theta = math.pi - theta # 180 - theta
        else:
            theta = math.pi + theta # 180 + theta
        
    return theta

def generate_data(num_samples):
    data = []
    labels = []
    for _ in range(num_samples):
        defense = create_defense()
        attack = create_attack(defense)
        
        # Use your existing calc_angle function
        theta = calc_angle(defense, attack)
        
        data.append(np.concatenate([defense, attack]))
        labels.append(theta)
    
    return np.array(data), np.array(labels)

# Generate data
X, y = generate_data(num_samples=1000000)

# Split the original dataset into a training set and a remaining set (80% train, 20% remaining)
X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the remaining data into validation and test sets (50% validation, 50% test of the remaining data)
X_val, X_test, y_val, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=42)


In [11]:
# Load the base model
base_model = tf.keras.models.load_model("defense_attack_angle_model.h5")
base_model.trainable = False

# Create a new model on top of the base model
inputs = tf.keras.Input(shape=(4,))
x = base_model(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)  # First new layer
x = tf.keras.layers.Dense(32, activation='relu')(x)  # Second new layer
outputs = tf.keras.layers.Dense(1)(x)

new_model = tf.keras.Model(inputs=inputs, outputs=outputs)


new_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [12]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the new model
history = new_model.fit(X_train, y_train,
                        epochs=100,
                        batch_size=64,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping])

Epoch 1/100
12500/12500 [==============================] - 46s 4ms/step - loss: 0.0306 - mae: 0.0380 - val_loss: 0.0169 - val_mae: 0.0358
Epoch 2/100
12500/12500 [==============================] - 26s 2ms/step - loss: 0.0174 - mae: 0.0320 - val_loss: 0.0166 - val_mae: 0.0290
Epoch 3/100
12500/12500 [==============================] - 29s 2ms/step - loss: 0.0174 - mae: 0.0313 - val_loss: 0.0166 - val_mae: 0.0298
Epoch 4/100
12500/12500 [==============================] - 45s 4ms/step - loss: 0.0174 - mae: 0.0310 - val_loss: 0.0165 - val_mae: 0.0275
Epoch 5/100
12500/12500 [==============================] - 63s 5ms/step - loss: 0.0173 - mae: 0.0309 - val_loss: 0.0171 - val_mae: 0.0380
Epoch 6/100
12500/12500 [==============================] - 65s 5ms/step - loss: 0.0173 - mae: 0.0306 - val_loss: 0.0164 - val_mae: 0.0276
Epoch 7/100
12500/12500 [==============================] - 61s 5ms/step - loss: 0.0173 - mae: 0.0306 - val_loss: 0.0166 - val_mae: 0.0290
Epoch 8/100
12500/12500 [=========

In [15]:
test_loss, test_mae = new_model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test MAE: {test_mae}")

predictions = new_model.predict(X_test)

for i in range(10):
    print(f"True angle: {y_test[i]:.6f}, Predicted angle: {predictions[i][0]:.6f}")

# Save the new model
new_model.save("defense_attack_angle_model_v2.h5")

3125/3125 [==============================] - 3s 889us/step - loss: 0.0153 - mae: 0.0261
Test Loss: 0.015329229645431042
Test MAE: 0.026079973205924034
3125/3125 [==============================] - 3s 835us/step
True angle: 1.065397, Predicted angle: 1.060499
True angle: 1.717918, Predicted angle: 1.751716
True angle: 4.567377, Predicted angle: 4.559901
True angle: 6.237886, Predicted angle: 6.198756
True angle: 4.843874, Predicted angle: 4.886951
True angle: 3.104582, Predicted angle: 3.040718
True angle: 0.725849, Predicted angle: 0.765149
True angle: 2.229640, Predicted angle: 2.220573
True angle: 5.720594, Predicted angle: 5.743846
True angle: 0.134659, Predicted angle: 0.156323
